In [44]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer


In [45]:
# Global variable area
model_performance_list = []


In [46]:
# There are three ways we can handle missing values in data science
# 1: Drop columns with missing values
# 2: Imputation
# 3 : Extension to imputation

def checkNull(dataset):
    isNull = dataset.isnull().any()
    flag = False
    for x in isNull:
        if x == True:
            flag = True
    if flag == True:
        print("This data set contains null values")
    else:
        print("Data set is clear and ready to work with")

def score_dataset(X_train,X_valid,y_train,y_valid):
    model = RandomForestRegressor(n_estimators=10,random_state = 0)
    model.fit(X_train,y_train)
    preds = model.predict(X_valid)
    
    return mean_absolute_error(y_valid, preds)
        
def drop_columns_with_missing_values(X_train,x_valid):
    cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
    
    # drops columns in training and validation data
    removed_x_train = X_train.drop(cols_with_missing,axis = 1)
    removed_x_valid = x_valid.drop(cols_with_missing,axis = 1)
    print("MAE from Approach 1 (Drop columns with missing values):")
    
    # Registering the performance data into the cache
    model_performance_list.append(score_dataset(removed_x_train, removed_x_valid, y_train, y_valid))
    
    print(score_dataset(removed_x_train, removed_x_valid, y_train, y_valid))

def imputer_style(X_train,x_valid):
#     print("This is from imputed dataset")
    imputer = SimpleImputer()
    
    imputed_x_train = pd.DataFrame(imputer.fit_transform(X_train))
    imputed_x_valid = pd.DataFrame(imputer.transform(x_valid))
    
    # Imputation removed column names; put them back
    imputed_x_train.columns = X_train.columns
    imputed_x_valid.columns = x_valid.columns
#     print(imputed_x_train.head())
    
    model_performance_list.append(score_dataset(imputed_x_train,imputed_x_valid,y_train,y_valid))
    print(score_dataset(imputed_x_train,imputed_x_valid, y_train,y_valid))
    

def ext_imputer_style(X_train,x_valid):
    imputer = SimpleImputer()
    
    cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
    
    # Make copy to avoid changing the dataset 
    x_train_plus = X_train.copy()
    x_valid_plus = x_valid.copy()
    
    # Make new columns indicating what will be imputed
    for col in cols_with_missing:
        x_train_plus[col+'_was_missing'] = X_train[col].isnull()
        x_valid_plus[col+'_was_missing'] = x_valid_plus[col].isnull()
        
    # Imputation
    imputed_x_train = pd.DataFrame(imputer.fit_transform(x_train_plus))
    imputed_x_valid = pd.DataFrame(imputer.transform(x_valid_plus))
    
    # The imputation process remove the column so ; put it back
    imputed_x_train.columns = x_train_plus.columns
    imputed_x_valid.columns = x_valid_plus.columns
        
    model_performance_list.append(score_dataset(imputed_x_train,imputed_x_valid,y_train,y_valid))
    print("MAE from Approach 3 (An Extension to Imputation):")
    print(score_dataset(imputed_x_train, imputed_x_valid, y_train, y_valid))

In [49]:
if __name__ == "__main__":
    data = pd.read_csv('melb_data.csv')
    y = data.Price
    melb_predictors = data.drop(['Price'],axis = 1)
    X = melb_predictors.select_dtypes(exclude=['object'])
    
#     checkNull(X)  # Checking if there is any null value exist in the dataset 
#     if :
#         print("This dataset got some null values")

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)
    
    drop_columns_with_missing_values(X_train,X_valid)
    imputer_style(X_train,X_valid)
    ext_imputer_style(X_train,X_valid)
    
    print("Shape of the data ",X_train.shape)
    
    print("Retriving cached performance data: ",model_performance_list)
    print("Among these values the best one is ",min(model_performance_list))
    model_performance_list.clear() # Clearing the cache for ignoring the extra values

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635
178166.46269899711
MAE from Approach 3 (An Extension to Imputation):
178927.503183954
Shape of the data  (10864, 12)
Retriving cached performance data:  [183550.22137772635, 178166.46269899711, 178927.503183954, 183550.22137772635, 178166.46269899711, 178927.503183954]
Among these values the best one is  178166.46269899711
